In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
tf.reset_default_graph()
test_input = tf.constant(np.zeros((10, 224, 224, 3)), dtype=tf.float32)
conv_1 = tf.layers.conv2d(test_input, filters=16, kernel_size=[5, 5], 
                                strides = 2, padding="same", activation=tf.nn.relu)
conv_2 = tf.layers.conv2d(conv_1, filters=8, kernel_size=[5, 5], 
                                strides = 2, padding="same", activation=tf.nn.relu)
conv_3 = tf.layers.conv2d(conv_2, filters=4, kernel_size=[5, 5], 
                                strides = 2, padding="same", activation=tf.nn.relu)
conv_4 = tf.layers.conv2d(conv_3, filters=2, kernel_size=[5, 5], 
                                strides = 2, padding="same", activation=tf.nn.relu)
conv_5 = tf.layers.conv2d(conv_4, filters=1, kernel_size=[5, 5], 
                                strides = 2, padding="same", activation=tf.nn.relu)

compression = tf.contrib.layers.flatten(conv_5)

deconv_5 = tf.layers.conv2d_transpose(conv_5, filters=2, kernel_size=[5, 5], 
                                strides = 2, padding="same", activation=tf.nn.relu)
deconv_4 = tf.layers.conv2d_transpose(deconv_5, filters=4, kernel_size=[5, 5], 
                                strides = 2, padding="same", activation=tf.nn.relu)
deconv_3 = tf.layers.conv2d_transpose(deconv_4, filters=8, kernel_size=[5, 5], 
                                strides = 2, padding="same", activation=tf.nn.relu)
deconv_2 = tf.layers.conv2d_transpose(deconv_3, filters=16, kernel_size=[5, 5], 
                                strides = 2, padding="same", activation=tf.nn.relu)
restored_image = tf.layers.conv2d_transpose(deconv_2, filters=3, kernel_size=[5, 5], 
                                strides = 2, padding="same", activation=tf.nn.relu)

print compression
print restored_image

In [ ]:
tf.trainable_variables()